# LDA with gensim

In [1]:
import json

In [2]:
# Generated by mass2lda
lda_dict = json.load(open('METABOLOMICS-SNETS-e9eaf6ec-download_clustered_spectra-main.lda.json'))
corpus = json.load(open('METABOLOMICS-SNETS-e9eaf6ec-download_clustered_spectra-main.corpus.json'))

In [3]:
K = 300
n_its = 1000

In [4]:
m = []
for doc, words in corpus.items():
    bow = []
    for word, score in words.items():
        bow.append((lda_dict['word_index'][word], score))
    m.append(bow)

In [5]:
from gensim.models.ldamulticore import LdaMulticore

In [6]:
%time lda = LdaMulticore(m, num_topics=K, iterations=n_its)

CPU times: user 14 s, sys: 2.36 s, total: 16.4 s
Wall time: 16.7 s


In [7]:
lda.get_topic_terms(3)

[(494, 0.13558426),
 (16, 0.12675428),
 (95, 0.0973422),
 (493, 0.091655105),
 (495, 0.091655105),
 (92, 0.09016258),
 (91, 0.058767717),
 (93, 0.056773014),
 (96, 0.046862844),
 (94, 0.042867154)]